<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLNN_func_approx_3D_110422.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# data

f=x1^3-0.2*x2+5

In [33]:
data=[
[0.32,	0.99,	0.77,	0.69,	0.04,	0.37,	0.25,	0.44,	0.42,	0.64,	0.8,],
[0.4,	0.23,	0.22,	0.57,	0.78,	0.69,	0.2,	0.45,	0.17,	0.98,	0.96,],
[4.952768,	5.924299,	5.412533,	5.214509,	4.844064,	4.912653,	4.975625,	4.995184,	5.040088,	5.066144,	5.32,],
]
data=np.array(data).T
data=pd.DataFrame(data, columns=['x1','x2', 'f'])#.sort_values(by=['x1'])
data.head()

,x1,x2,f
0,0.32,0.40,4.952768
1,0.99,0.23,5.924299
2,0.77,0.22,5.412533
3,0.69,0.57,5.214509
4,0.04,0.78,4.844064


In [109]:
fig_data=go.Figure()
fig_data.add_trace(go.Scatter3d(x=data.x1, y=data.x2, z=data.f, mode='markers'))
x=np.linspace(0, 1, 11)
y=np.linspace(0, 1, 11)
x, y = np.meshgrid(x,y)
z=x**3-.2*y+5
fig_data.add_trace(go.Surface(x=x, y=y, z=z))
fig_data.show()

# scaling

In [35]:
pip install MLNN

In [36]:
from sklearn import preprocessing
from MLNN import *

In [67]:
scaler_X = preprocessing.StandardScaler().fit(data[['x1','x2']])
print(scaler_X.mean_, scaler_X.scale_)
X_sc = scaler_X.transform(data[['x1','x2']]) # matrix
X_sc[0:3]

[0.52090909 0.51363636] [0.26752925 0.28952226]


array([[-0.7509799 , -0.39249612],
       [ 1.75341913, -0.97967031],
       [ 0.93107915, -1.01420996]])

In [38]:
scaler_y = preprocessing.StandardScaler().fit(data[['f']])
y_sc = scaler_y.transform(data[['f']])[:,0] # vector
y_sc

array([-0.66871956,  2.61337708,  0.88449198,  0.21551291, -1.03595131,
       -0.80423897, -0.59150238, -0.52542674, -0.37372879, -0.28570451,
        0.57189028])

# model fitting

In [229]:
layers=[2,2,1]

In [230]:
reg1=Regres(layers=layers)
reg1.set_hidden_activ('mrelu', param=0.01)
reg1.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.01)
data1=go.Scatter(y=reg1.MSE_history[:], name='eta=0.01')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [231]:
reg2=Regres(layers=layers)
reg2.set_hidden_activ('mrelu', param=0.01)
reg2.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.001)
data2=go.Scatter(y=reg2.MSE_history[:], name='eta=0.001')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [232]:
reg_fixed=Regres(layers=layers)
reg_fixed.set_hidden_activ('mrelu', param=0.01)
# reg_fixed.set_weights(fixed_weights) # if no fixed_weights exist yet, skip it
reg_fixed.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.0001)
data_fixed=go.Scatter(y=reg_fixed.MSE_history[:], name='fixed')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [233]:
fig1=go.Figure()
fig1.add_trace(data1), print('MSE1=', reg1.MSE_history[-1])
fig1.add_trace(data2), print('MSE2=', reg2.MSE_history[-1])

fig1.add_trace(data_fixed), print('MSE_fixed=', reg_fixed.MSE_history[-1])

fig1.update_yaxes(range=[0, 1])
fig1.show()

MSE1= 0.03645419940739201
MSE2= 0.02755858074295775
MSE_fixed= 1.334325477998955


# model fixation

In [234]:
reg1.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.56446122 -0.16142885 -0.41573885] mrelu
Layer 1 Neuron 1 : [-0.91917005  0.26932202 -0.0269946 ] mrelu
Layer 2 Neuron 0 : [ 1.18817235 -0.33208351 -0.45033722] linear



In [235]:
reg2.weight_history[-1]

[[[1.4106222508530821, -0.15216369611325525, -0.5251792157211558],
  [-0.38777417797905417, 0.0861523982455459, 0.7642008900347037]],
 [[1.1066013204408893, -0.8373151441213467, 0.2276434534999552]]]

In [236]:
fixed_weights=[[[1.4106222508530821, -0.15216369611325525, -0.5251792157211558],
  [-0.38777417797905417, 0.0861523982455459, 0.7642008900347037]],
 [[1.1066013204408893, -0.8373151441213467, 0.2276434534999552]]]
fixed_MSE=0.02755858074295775

# predictions

In [246]:
# chosen_model=reg_fixed
chosen_model=reg2

In [247]:
predics=scaler_y.inverse_transform([chosen_model.predict(np.array(X_sc))])[0]
errors=(predics-data.f)/data.f*100

## mesh

In [248]:
pip install doepy

In [249]:
from doepy import build

In [250]:
list1={'x1':np.linspace(0, 1, 11),'x2':np.linspace(0, 1, 11)}
mesh_data=build.full_fact(list1)
display(mesh_data[:5])

,x1,x2
0,0.0,0.0
1,0.1,0.0
2,0.2,0.0
3,0.3,0.0
4,0.4,0.0


In [251]:
mesh_sc=scaler_X.transform(mesh_data[['x1','x2']])
mesh_predics=scaler_y.inverse_transform([chosen_model.predict(np.array(mesh_sc))])[0]

## plot

In [252]:
fig_predic=go.Figure()
fig_predic.add_trace(go.Scatter3d(x=data.x1, y=data.x2, z=data.f, mode='markers', name='data'))
fig_predic.add_trace(go.Scatter3d(x=data.x1, y=data.x2, z=predics, mode='markers', name='predics'))
fig_predic.add_trace(go.Scatter3d(x=mesh_data.x1, y=mesh_data.x2, z=mesh_predics, mode='markers', marker_size=2, name='mesh-predics'))
x=np.linspace(0, 1, 11)
y=np.linspace(0, 1, 11)
x, y = np.meshgrid(x,y)
z=x**3-.2*y+5
fig_predic.add_trace(go.Surface(x=x, y=y, z=z, colorbar_x=0, opacity=0.5,))
# fig_predic.update_scenes(camera_projection_type="orthographic")

fig_predic.show()

In [253]:
chosen_model.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [ 1.41062225 -0.1521637  -0.52517922] mrelu
Layer 1 Neuron 1 : [-0.38777418  0.0861524   0.76420089] mrelu
Layer 2 Neuron 0 : [ 1.10660132 -0.83731514  0.22764345] linear



In [254]:
print('X mean=', scaler_X.mean_,',X stdev=', scaler_X.scale_)
print('y mean=', scaler_y.mean_[0],',y stdev=', scaler_y.scale_[0])

X mean= [0.52090909 0.51363636] ,X stdev= [0.26752925 0.28952226]
y mean= 5.150715181818182 ,y stdev= 0.29600926114591214
